In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.covariance import LedoitWolf
import warnings
import cvxpy as cp

# ------ Institutional Configuration ------
CONFIG = {
    "data": {
        "min_volatility": 0.005,
        "max_volatility": 0.30,
        "max_correlation": 0.85,
        "min_history": 60,
        "min_assets": 50
    },
    "optimization": {
        "max_weight": 0.05,
        "min_weight": 0.005,
        "sector_limit": 0.25,
        "risk_parity_tol": 0.15
    },
    "liquidity": {
        "adv_threshold": 1e6
    }
}

DATA_PATH = Path('./data')
OUTPUT_PATH = DATA_PATH / 'outputs' 
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# ------ Robust Data Pipeline ------
def load_and_preprocess_data():
    """Quant-grade data processing with adaptive filtering"""
    print("📊 Loading and scrubbing data...")
    try:
        df = pd.read_parquet(DATA_PATH / 'processed/features_with_regime.parquet')
        df = df.sort_values(['date', 'ticker'])
        
        # Clean and deduplicate
        df = df.drop_duplicates(subset=['date', 'ticker'], keep='last')
        df['returns_1d'] = np.clip(df['returns_1d'], -0.15, 0.15)
        
        # Pivot and filter
        returns = df.pivot(index='date', columns='ticker', values='returns_1d')
        returns = returns.ffill(limit=3).bfill(limit=3)
        
        # Liquidity screen
        returns = returns.dropna(axis=1, thresh=CONFIG['data']['min_history'])
        returns = returns.dropna(how='all', axis=0)
        
        if returns.empty:
            raise ValueError("No data remaining after initial cleaning")
            
        # Volatility filter
        vol = returns.std()
        valid_vol = vol.between(CONFIG['data']['min_volatility'], 
                              CONFIG['data']['max_volatility'])
        returns = returns.loc[:, valid_vol]
        
        if returns.shape[1] < CONFIG['data']['min_assets']:
            raise ValueError("Insufficient assets after volatility filtering")
            
        # Adaptive correlation pruning
        current_threshold = CONFIG['data']['max_correlation']
        for _ in range(3):
            corr_matrix = returns.corr().abs()
            mask = np.triu(np.ones(corr_matrix.shape, dtype=bool), k=1)
            upper = corr_matrix.where(mask)
            
            to_drop = [col for col in upper.columns 
                      if (upper[col] > current_threshold).any()]
            returns = returns.drop(columns=to_drop)
            
            if returns.shape[1] >= CONFIG['data']['min_assets']:
                break
            current_threshold *= 1.05
        
        returns = returns.dropna(how='any', axis=0)
        if returns.shape[1] < 10:
            raise ValueError("Insufficient assets for diversification")
            
        print(f"✅ Clean matrix: {returns.shape}")
        return returns.astype(np.float32)
    
    except Exception as e:
        print(f"🚨 Data pipeline failure: {str(e)}")
        raise

# ------ Portfolio Construction Engine ------
class PortfolioOptimizer:
    """Institutional portfolio builder with robust risk management"""
    
    def __init__(self, returns):
        self.returns = returns
        self.tickers = returns.columns.tolist()
        self.n_assets = len(self.tickers)
        self.S = self._robust_covariance()
        self.asset_vol = np.sqrt(np.diag(self.S))
        self.L = np.linalg.cholesky(self.S.values + 1e-3*np.eye(self.n_assets))  # Regularized Cholesky
        
    def _robust_covariance(self):
        """Regularized covariance matrix with eigenvalue clipping"""
        lw = LedoitWolf(assume_centered=True).fit(self.returns)
        cov = lw.covariance_ * 252
        cov += np.eye(cov.shape[0]) * 1e-3  # Enhanced regularization
        return pd.DataFrame(cov, index=self.tickers, columns=self.tickers)
    
    def _get_feasible_bounds(self):
        """Dynamic weight constraints with feasibility checks"""
        config_min = CONFIG['optimization']['min_weight']
        config_max = CONFIG['optimization']['max_weight']
        n = self.n_assets
        
        # Initial bounds
        min_w = max(config_min, 1/(n*3))
        max_w = min(config_max, 3/n)
        
        # Feasibility checks
        min_sum = min_w * n
        if min_sum > 1:
            min_w = 1/(n*3)  # Relax minimum constraint
        max_sum = max_w * n
        if max_sum < 1:
            max_w = min(3/n, 0.5)  # Relax maximum constraint
            
        return min_w, max_w
    
    def max_diversification(self):
        """Diversification ratio via SOCP formulation"""
        try:
            min_w, max_w = self._get_feasible_bounds()
            
            w = cp.Variable(self.n_assets)
            t = cp.Variable()
            
            objective = cp.Maximize(t)
            constraints = [
                w >= min_w,
                w <= max_w,
                cp.sum(w) == 1,
                w @ self.asset_vol >= t * cp.norm(self.L.T @ w)
            ]
            
            prob = cp.Problem(objective, constraints)
            prob.solve(solver=cp.ECOS, max_iters=5000)
            
            if prob.status not in ["optimal", "optimal_inaccurate"]:
                raise RuntimeError(f"Solve status: {prob.status}")
                
            weights = pd.Series(w.value, index=self.tickers).clip(0)
            weights = weights[weights >= 1e-4]
            return weights / weights.sum()
            
        except Exception as e:
            print(f"⚠️ Diversification failed: {str(e)}")
            return pd.Series()

    def min_volatility(self):
        """Convex minimum volatility implementation"""
        try:
            min_w, max_w = self._get_feasible_bounds()
            
            w = cp.Variable(self.n_assets)
            risk = cp.quad_form(w, self.S.values)
            
            constraints = [
                w >= min_w,
                w <= max_w,
                cp.sum(w) == 1
            ]
            
            prob = cp.Problem(cp.Minimize(risk), constraints)
            prob.solve(solver=cp.ECOS, max_iters=5000)  # Switched to ECOS
            
            if prob.status not in ["optimal", "optimal_inaccurate"]:
                raise RuntimeError(f"Solve status: {prob.status}")
                
            weights = pd.Series(w.value, index=self.tickers).clip(0)
            weights = weights[weights >= 1e-4]
            return weights / weights.sum()
            
        except Exception as e:
            print(f"⚠️ Min volatility failed: {str(e)}")
            return pd.Series()

# ------ Risk Management System ------
class RiskManager:
    """Advanced risk management framework"""
    
    def __init__(self, sector_map, adv_data):
        self.sector_map = sector_map
        self.adv_data = adv_data
        
    def liquidity_adjustment(self, weights):
        """ADV-based liquidity scaling with fallback"""
        try:
            adv_ratio = np.sqrt(self.adv_data[weights.index] / CONFIG['liquidity']['adv_threshold'])
            scaling = np.clip(adv_ratio, 0.2, 5.0)
            scaled = weights * scaling
            scaled[scaled < 1e-4] = 0
            
            total = scaled.sum()
            if total < 1e-6:
                raise ValueError("All weights eliminated in liquidity adjustment")
                
            return scaled / total
            
        except Exception as e:
            print(f"⚠️ Liquidity adjustment failed: {str(e)}")
            return weights

    def sector_constraints(self, weights):
        """Adaptive sector constraints with progressive relaxation"""
        try:
            sector_exposure = weights.groupby(self.sector_map[weights.index]).sum()
            over_limit = sector_exposure > CONFIG['optimization']['sector_limit']
            
            if not over_limit.any():
                return weights
                
            # Progressive relaxation logic
            max_relaxation = 3.0
            step = 0.1
            current_relaxation = 1.0
            sector_limits = {}
            
            while current_relaxation <= max_relaxation:
                sector_limits = {
                    s: min(CONFIG['optimization']['sector_limit'] * current_relaxation, 0.6)
                    for s in sector_exposure.index
                }
                
                total_required = sum(min(sector_exposure[s], limit) 
                                 for s, limit in sector_limits.items())
                if total_required >= 0.99:
                    break
                
                current_relaxation += step
                
            print(f"⚠️ Relaxing sector limits to {current_relaxation:.1f}x original")
            
            # Quadratic programming with fallback
            w = cp.Variable(len(weights))
            constraints = [
                cp.sum(w) == 1,
                w >= 0,
                w <= CONFIG['optimization']['max_weight']
            ]
            
            for sector, limit in sector_limits.items():
                sector_mask = self.sector_map[weights.index] == sector
                constraints.append(cp.sum(w[sector_mask]) <= limit)
            
            prob = cp.Problem(cp.Minimize(cp.sum_squares(w - weights.values)), constraints)
            prob.solve(solver=cp.ECOS, max_iters=5000)
            
            if prob.status not in ["optimal", "optimal_inaccurate"]:
                print("⚠️ Sector QP failed, using proportional scaling")
                scaled_weights = weights.copy()
                for sector, limit in sector_limits.items():
                    sector_mask = self.sector_map[weights.index] == sector
                    sector_sum = scaled_weights[sector_mask].sum()
                    if sector_sum > limit:
                        scaled_weights[sector_mask] *= (limit / sector_sum)
                return scaled_weights / scaled_weights.sum()
                
            adjusted = pd.Series(w.value, index=weights.index).clip(0)
            adjusted /= adjusted.sum()
            return adjusted[adjusted >= 1e-4]
            
        except Exception as e:
            print(f"⚠️ Sector constraints failed: {str(e)}")
            return weights

# ------ Portfolio Manager ------
class PortfolioManager:
    """Institutional portfolio management system"""
    
    def __init__(self, returns, sector_map, adv_data):
        self.returns = returns
        self.sector_map = sector_map
        self.adv_data = adv_data
        self.optimizer = PortfolioOptimizer(returns)
        self.risk_mgr = RiskManager(sector_map, adv_data)
        
    def build_portfolio(self):
        """Robust portfolio construction pipeline"""
        try:
            # Attempt 1: Max Diversification
            weights = self.optimizer.max_diversification()
            
            # Attempt 2: Min Volatility
            if weights.empty:
                print("⚠️ Falling back to minimum volatility")
                weights = self.optimizer.min_volatility()
            
            # Risk management
            weights = self.risk_mgr.liquidity_adjustment(weights)
            weights = self.risk_mgr.sector_constraints(weights)
            
            # Final cleanup
            valid_weights = weights[weights >= 1e-4]
            if valid_weights.empty:
                raise ValueError("All weights below minimum threshold")
                
            return valid_weights / valid_weights.sum()
            
        except Exception as e:
            print(f"⚠️ Portfolio construction failed: {str(e)}")
            return self._equal_weight_fallback()
    
    def _equal_weight_fallback(self):
        """Equal weight final fallback"""
        n = len(self.optimizer.tickers)
        weights = pd.Series(1/n, index=self.optimizer.tickers)
        return weights[weights >= 1e-4]

# ------ Execution ------
if __name__ == "__main__":
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        try:
            returns = load_and_preprocess_data()
            sector_map = pd.Series(
                index=returns.columns,
                data=np.random.choice(['Tech', 'Healthcare', 'Energy'], len(returns.columns))
            )
            adv_data = pd.Series(
                index=returns.columns,
                data=np.abs(np.random.lognormal(18, 0.3, len(returns.columns)))
            )
            
            pm = PortfolioManager(returns, sector_map, adv_data)
            weights = pm.build_portfolio()
            
            if not weights.empty:
                print("\n📊 Final Portfolio Composition:")
                top10 = weights.sort_values(ascending=False).head(10)
                print(top10.apply(lambda x: f"{x:.2%}"))
                pd.DataFrame(weights, columns=['weight']).to_csv(
                    OUTPUT_PATH / 'portfolio_weights.csv'
                )
            else:
                print("⚠️ No valid portfolio constructed")
                
        except Exception as e:
            print(f"🚨 Critical failure: {str(e)}")

📊 Loading and scrubbing data...
✅ Clean matrix: (59, 465)
⚠️ Diversification failed: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
var7170 @ Pnorm([[0.72 0.02 ... 0.10 0.07]
 [0.00 0.82 ... -0.05 -0.01]
 ...
 [0.00 0.00 ... 0.51 -0.00]
 [0.00 0.00 ... 0.00 0.41]] @ var7169, 2) <= var7169 @ [0.71574065 0.81919478 0.70909517 0.40787027 0.75412841 0.54689035
 1.15798655 0.68002973 0.65990317 0.7414002  0.60385705 1.05362151
 0.48724073 0.50240482 0.88389859 0.493214   0.53356333 0.77240693
 0.73367179 0.49703856 0.54909239 0.59307509 0.4258729  0.66246326
 1.15384296 0.52680052 0.60273139 0.78769163 1.04297685 0.57640546
 0.81687218 0.45217778 1.31872082 0.58859385 1.1441885  1.20839212
 1.39514269 0.76617457 0.49696864 0.59205536 0.58318317 0.62075786
 0.64224963 0.96046719 0.5397311  0.99441102 0.48715727 0.41784376
 0.41548696 0.70191669 0.91143592 0.77570906 0.4998145  0.52271885
 0.59086991 0.60616369 0.70105167 0.70179184 0.48801905 0.839551